In [ ]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [ ]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

57344/57026 [==============================] - 0s 2us/step


In [ ]:
print (train_data.shape, train_targets.shape, test_data.shape, test_targets.shape)

(404, 13) (404,) (102, 13) (102,)


In [ ]:
print (train_data[0], train_targets[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ] 15.2


In [ ]:
print (train_targets[0])

15.2


여기서 볼 수 있듯이 404개의 훈련 샘플과 102개의 테스트 샘플이 있고 모두 13개의 수치 특성을 가지고 있습니다. 13개의 특성은 다음과 같습니다:

1. Per capita crime rate.
2. Proportion of residential land zoned for lots over 25,000 square feet.
3. Proportion of non-retail business acres per town.
4. Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
5. Nitric oxides concentration (parts per 10 million).
6. Average number of rooms per dwelling.
7. Proportion of owner-occupied units built prior to 1940.
8. Weighted distances to five Boston employment centres.
9. Index of accessibility to radial highways.
10. Full-value property-tax rate per $10,000.
11. Pupil-teacher ratio by town.
12. 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
13. % lower status of the population.

타깃은 주택의 중간 가격으로 천달러 단위입니다:

Data Normalization 

In [ ]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [ ]:
mean

array([3.74511057e+00, 1.14801980e+01, 1.11044307e+01, 6.18811881e-02,
       5.57355941e-01, 6.26708168e+00, 6.90106436e+01, 3.74027079e+00,
       9.44059406e+00, 4.05898515e+02, 1.84759901e+01, 3.54783168e+02,
       1.27408168e+01])

In [ ]:
print (train_data[0], train_targets[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ] 15.2


In [ ]:
from keras import models
from keras import layers

def build_model():
    # 동일한 모델을 여러 번 생성할 것이므로 함수를 만들어 사용합니다
    model = models.Sequential()
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

마지막 레어에는 Activation function이 없습니다. 회귀 문제(어떤 값을 예측하는 문제)를 풀때 이와 같이 구성합니다.
평균 제곱 오차 (mean squared error)
절대 평균 오차 (mean absolute error)

0.5 indicate $500

**K-Fold Validation**

In [ ]:
import numpy as np

k = 4

num_val_samples = len(train_data) // k
num_epochs = 10
all_scores = []
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]
        ], axis=0)

    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=10, verbose=1)
    
    # 검증 세트로 모델 평가
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

처리중인 폴드 # 0
Epoch 1/10
303/303 [==============================] - 2s 6ms/step - loss: 499.0270 - mae: 20.4416
Epoch 2/10
303/303 [==============================] - 0s 277us/step - loss: 331.6504 - mae: 16.0167
Epoch 3/10
303/303 [==============================] - 0s 274us/step - loss: 178.0961 - mae: 10.6489
Epoch 4/10
303/303 [==============================] - 0s 292us/step - loss: 95.7569 - mae: 7.6571
Epoch 5/10
303/303 [==============================] - 0s 293us/step - loss: 66.5982 - mae: 6.2317
Epoch 6/10
303/303 [==============================] - 0s 281us/step - loss: 56.1244 - mae: 5.6615
Epoch 7/10
303/303 [==============================] - 0s 277us/step - loss: 45.0523 - mae: 5.0580
Epoch 8/10
303/303 [==============================] - 0s 333us/step - loss: 48.0588 - mae: 5.2807
Epoch 9/10
303/303 [==============================] - 0s 309us/step - loss: 33.8512 - mae: 4.3538
Epoch 10/10
303/303 [==============================] - 0s 301us/step - loss: 37.8146 - mae: 4.6057
처리중

In [ ]:
all_scores

[3.482288122177124, 3.9042980670928955, 3.8140201568603516, 3.5640647411346436]

In [ ]:
np.mean(all_scores)

3.6911677718162537

In [ ]:
# 아래 모델은 k fold cross validation에서 마지막에 훈련된 모델입니다.
# 본 실습에서 별도의 test data set을 구성하지 않았기 때문에, vadliation data set을 이용해 house price 를 예측prediction 해보겠습니다.

predictedHousePrice = model.predict(val_data)

In [ ]:
# validation set 은 100개의 데이터 셋으로 구성되어 있습니다만 아래와 같이 1대1 비교를 해보겠습니다.
# val_data[0] 에 대한 예측값 predictedHousePrice[0]
# val_data[0] 에 대한 실제값 val_targets[0]
print (predictedHousePrice[0], val_targets[0])

[28.648996] 36.2


In [ ]:
new_data = [1.47, 0.0, 8.14, 0.0, 0.53, 6.1, 91.7, 4.9769,  4.12,  307.6 ,  22,  296.9, 18.72]

In [ ]:
new_data

[1.47, 0.0, 8.14, 0.0, 0.53, 6.1, 91.7, 4.9769, 4.12, 307.6, 22, 296.9, 18.72]

In [ ]:
new_data -= mean
new_data /= std

In [ ]:
print (new_data.shape)

(13,)


In [ ]:
new_data = np.asarray(new_data)

In [ ]:
new_data = np.expand_dims(new_data, axis=0)

In [ ]:
print (new_data.shape)

(1, 13)


In [ ]:
model.predict(new_data)

array([[12.910594]], dtype=float32)